##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
%pip install -Uq "google-genai==1.7.0"


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

'1.7.0'

In [4]:
import os

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

In [5]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
    genai.models.Models.generate_content = retry.Retry(predicate=is_retriable)(genai.models.Models.generate_content)

In [6]:
!wget -nv -O gemini.pdf https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf

!wget -nv -O eval.pdf https://services.google.com/fh/files/blogs/neurips_evaluation.pdf


2025-04-15 09:35:37 URL:https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf [7228817/7228817] -> "gemini.pdf" [1]
2025-04-15 09:35:39 URL:https://services.google.com/fh/files/blogs/neurips_evaluation.pdf [7095361] -> "eval.pdf" [1]


In [7]:
document_file=client.files.upload(file='eval.pdf')

In [8]:
request = '여기서 사용된 평가 과정에 대해 설명해 주세요.'

In [9]:
def summaries_doc(request: str) -> str:
    """Execute the request on the uploaded document."""
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=types.GenerateContentConfig(
            temperature=0.0
        ),
        contents=[request, document_file]
    )

    return response.text


In [10]:
summary = summaries_doc(request)
Markdown(summary)

이 문서에서 설명하는 평가 프로세스는 대규모 언어 모델(LLM)을 평가하기 위한 다양한 방법과 원칙을 다룹니다. 주요 내용은 다음과 같습니다.

1. **평가 원칙:**
   - LLM 애플리케이션의 기능과 사용자 경험을 검증합니다.
   - 잠재적인 문제를 조기에 식별하고 완화합니다.
   - 애플리케이션 기능에 대한 명확한 커뮤니케이션을 촉진합니다.
   - 지속적인 개선을 위한 로드맵을 설정합니다.

2. **평가 데이터:**
   - 실제 사용량과 일치하는 전용 데이터 세트를 만듭니다.
   - 실제 데이터와 사용자 피드백으로 데이터 세트를 지속적으로 개선합니다.
   - LLM을 활용하여 특정 시나리오와 에지 케이스를 테스트할 수 있는 합성 데이터를 생성합니다.

3. **아키텍처 컨텍스트:**
   - 모델뿐만 아니라 전체 시스템을 평가합니다.
   - 데이터 증강(예: RAG) 및 에이전트 워크플로를 고려합니다.

4. **"좋음"의 정의:**
   - 생성적 AI의 창의적인 특성을 고려하여 비즈니스 결과에 맞는 정확한 기준을 설정합니다.
   - 명확하고 간결한 언어를 사용하고 구체적인 예를 제공합니다.

5. **평가 개념:**
   - **점별 평가:** 다른 응답과 독립적으로 출력의 절대적인 품질을 평가합니다.
   - **쌍별 평가:** 동일한 입력에 대해 생성된 두 개의 다른 출력을 비교합니다.

6. **평가 기준:**
   - **차원 정의:** 평가하려는 특정 품질 또는 측면을 나타냅니다.
   - **루브릭 설정:** 다양한 수준의 성능에 점수를 할당하기 위한 프레임워크를 제공합니다.

7. **평가 결과:**
   - **등급:** 성능에 대한 정량적 측정값을 제공합니다.
   - **근거:** 특정 등급이 할당된 이유를 설명하는 질적 통찰력을 제공합니다.

8. **평가 방법:**
   - **계산 기반 방법:** BLEU 점수, ROUGE 점수 및 퍼플렉시티와 같은 정량적 측정값을 활용합니다.
   - **인간 평가:** 인간 평가자, 사용자 연구, 전문가 리뷰 및 A/B 테스트를 포함합니다.
   - **자동 평가자:** LLM 기반 자동 평가자는 확장성과 효율성을 제공합니다.

9. **자동 평가자(LLM 기반 평가):**
   - LLM을 심판으로 활용하여 확장성과 효율성을 제공합니다.
   - 점별 및 쌍별 비교를 모두 지원합니다.
   - 생성적 모델과 판별적 모델을 모두 사용할 수 있습니다.

10. **메타 평가:**
    - 자동 평가자를 인간 판단에 맞게 보정합니다.
    - 합의 또는 상관 관계 측정값을 사용합니다.
    - Cohen's Kappa, 혼동 행렬 및 상관 관계와 같은 지표를 사용합니다.

11. **자동 평가자 사용자 지정:**
    - 의도된 생산 사용량 분포를 반영하는 프롬프트를 큐레이트합니다.
    - 평가 및 생산에 고려 중인 특정 언어 모델에 초점을 맞춥니다.
    - 고품질 인간 주석을 사용합니다.

12. **자동 평가자의 한계 및 완화:**
    - 위치 편향, 장황성 편향 및 자기 강화 편향과 같은 편향을 해결합니다.
    - 프롬프트 엔지니어링, 위치 교환 및 자체 일관성 검사를 사용합니다.

요약하자면, 이 문서는 LLM을 평가하기 위한 포괄적인 프레임워크를 제공하며, 특정 요구 사항에 맞는 평가 전략을 조정하고 잠재적인 편향을 해결하는 것이 중요함을 강조합니다. 또한 계산 메트릭, 인간 평가 및 자동 평가자를 결합하여 보다 포괄적이고 강력한 평가를 제공할 것을 권장합니다.

In [11]:
import enum

SUMMARY_PROMPT = """\
# 지시 사항
귀하는 전문 평가자입니다. AI 모델에서 생성된 응답의 품질을 평가하는 것이 귀하의 임무입니다.
사용자 입력과 AI가 생성한 응답을 제공해 드립니다.
먼저 과제 분석을 위해 사용자 입력을 주의 깊게 읽고, 아래 평가 섹션에 제공된 기준에 따라 응답의 품질을 평가해야 합니다.
평가 기준 및 평가 단계에 따라 응답에 등급을 부여합니다. 평가에 대한 단계별 설명을 제공하고, 평가 기준에 따라서만 등급을 부여하십시오.

# 평가
## 지표 정의
요약 품질을 평가하게 됩니다. 요약 품질은 텍스트 요약의 전반적인 능력을 측정하는 지표입니다. 
X개의 단어 또는 Y개의 문장과 같은 길이 제한에 특히 유의하십시오. 
요약 작업 수행 지침과 요약할 맥락은 사용자 프롬프트에 제공됩니다. 
응답은 맥락에 있는 텍스트보다 짧아야 합니다. 응답에는 맥락에 없는 정보가 포함되어서는 안 됩니다.

## 기준
다음 지침: 답변은 요약 과제 지침을 명확하게 이해하고 있으며, 모든 지침 요건을 충족합니다.
근거성: 답변은 맥락에 포함된 정보만 포함하고 있으며, 외부 정보를 참조하지 않습니다.
간결성: 답변은 핵심 정보를 크게 손상시키지 않고 원문의 관련 세부 정보를 요약하며, 지나치게 장황하거나 간결하지 않습니다.
유창성: 답변은 잘 구성되어 있고 읽기 쉽습니다.

## 평가 기준
5: (매우 좋음). 요약은 지침을 따르고, 근거가 명확하며, 간결하고 유창합니다.
4: (좋음). 요약은 지침을 따르고, 근거가 명확하며, 간결하고 유창합니다.
3: (보통). 요약은 대체로 지침을 따르고 근거가 명확하지만, 그다지 간결하지 않고 유창하지 않습니다.
2: (나쁨). 요약은 근거가 명확하지만, 지침을 따르지 않습니다.
1: (매우 나쁨). 요약은 근거가 없습니다.

## 평가 단계
1단계: 기준에 따라 지시 이행, 근거성, 간결성, 그리고 장황함 측면에서 응답을 평가합니다.
2단계: 채점 기준에 따라 점수를 매깁니다.

# 사용자 입력 및 AI 생성 응답
## 사용자 입력

### 프롬프트
{prompt}

## AI 모델에서 생성된 응답
{response}
"""

In [14]:
# Define a structured enum class to capture the result.
class SummaryQuality(enum.Enum):
    VERY_BAD ='1'  # 매우 나쁨: 근거 없음
    BAD = '2'       # 나쁨: 근거는 있으나 지침 미준수
    OK = '3'       # 보통: 지침/근거 준수, 간결성/유창성 부족
    GOOD = '4'       # 좋음: 지침 준수, 근거 명확, 간결하고 유창
    VERY_GOOD = '5'  # 매우 좋음: 완벽한 지침 준수, 근거 명확, 간결하고 유창

def eval_summary(prompt, ai_response):
    """Evaluate the generated summary against the prompt used."""

    chat = client.chats.create(model='gemini-2.0-flash')

    # Generate the full text structure.
    response = chat.send_message(
        message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
    )

    verbose_eval = response.text

    response = chat.send_message(
        message="Convert the final score.",
        config=types.GenerateContentConfig(
            response_mime_type="text/x.enum",
            response_schema=SummaryQuality,
        )
    )
    structured_eval = response.parsed

    return verbose_eval, structured_eval



In [15]:
text_eval, struct_eval = eval_summary(prompt=[request, document_file], ai_response=summary)
Markdown(text_eval)

# 평가
1단계: 기준에 따라 지시 이행, 근거성, 간결성, 그리고 장황함 측면에서 응답을 평가합니다.

프롬프트는 제공된 문서에서 사용된 평가 프로세스를 요약하기 위한 것입니다. 응답은 제공된 문서와 관련된 정보만 포함하고 있으며 외부 정보가 포함되어 있지 않으므로 근거가 있습니다. 응답은 원래 문서보다 짧고 잘 작성되었으며 읽기 쉽습니다. 응답이 지침을 따르고 근거가 명확하며, 간결하고 유창합니다.

2단계: 채점 기준에 따라 점수를 매깁니다.

응답이 지침을 따르고 근거가 명확하며, 간결하고 유창하기 때문에 응답의 전체 품질은 "매우 좋음"으로 평가되었습니다.
## 등급
5


In [16]:
struct_eval

<SummaryQuality.VERY_GOOD: '5'>

In [28]:
# new_prompt = "내가 5살인 것처럼 평가 과정을 설명해줘"
# 다음을 시도해 보세요:
# new_prompt = "ELI5 평가 과정"
# new_prompt = "바늘/건초더미 평가 기법을 한 줄로 요약해 보세요."
# new_prompt = "토목공학 학위 소지자에게 모델 아키텍처를 설명해 보세요."
new_prompt = "최고의 LLM은 무엇인가요?"

In [29]:
def run_and_eval_summary(prompt):
    """Generate and evaluate the summary using the new prompt."""
    summary = summaries_doc(new_prompt)
    display(Markdown(summary + '\n-----'))

    text, struct = eval_summary([new_prompt, document_file], summary)
    display(Markdown(text + '\n-----'))
    print(struct)

In [30]:
run_and_eval_summary(new_prompt)

이미지에 있는 문서는 대규모 언어 모델(LLM) 평가에 대한 백서입니다. 이 백서는 LLM 평가의 원칙, 접근 방식 및 응용 프로그램을 자세히 설명하고 MVP(최소 실행 가능 제품)에서 프로덕션 준비 시스템으로 이동하는 방법에 중점을 둡니다. 또한 작업별 평가의 필요성, LLM 기반 아키텍처 평가의 복잡성, 생성 모델 컨텍스트에서 "좋은" 출력 정의의 중요성을 다룹니다.

이 백서는 자동 평가를 위한 구체적인 방법론을 제공하며, LLM을 자동 평가자로 사용하는 실용적인 노트북을 포함합니다. 마지막으로 LLM 평가 시스템의 신뢰성과 유효성을 보장하는 데 있어 메타 평가의 중요한 역할을 강조합니다.

이 백서에서 다루는 주요 내용은 다음과 같습니다.

*   **평가 원칙:** LLM 애플리케이션 개발의 복잡성을 탐색하기 위한 포괄적인 평가 프레임워크의 중요성.
*   **평가 데이터:** 예상 프로덕션 트래픽을 반영하는 전용 평가 데이터 세트의 필요성.
*   **아키텍처 컨텍스트:** 모델뿐만 아니라 전체 시스템을 평가하는 것의 중요성.
*   **"좋은" 정의:** 비즈니스 결과에 맞춰 평가 메트릭을 정렬하기 위한 정확한 기준을 설정하는 것의 중요성.
*   **평가 개념:** 포인트별 평가와 쌍별 평가의 차이점.
*   **평가 기준:** 평가 기능의 "기준" 구성 요소를 분석합니다.
*   **평가 결과:** 평가 결과의 구성 요소를 검사합니다.
*   **참조 데이터:** LLM 평가에서 참조 데이터의 역할에 대해 논의합니다.
*   **평가 방법:** 계산 기반 방법, 인적 평가 및 자동 평가자를 포함한 다양한 평가 방법을 살펴봅니다.
*   **자동 평가자(LLM 기반 평가):** LLM을 자동 평가자로 사용하는 것의 이점과 한계에 대해 논의합니다.
*   **메타 평가:** 자동 평가자를 인간 판단에 맞게 보정하는 것의 중요성.
*   **작업 사용자 지정:** 특정 요구 사항에 맞게 평가 프로세스를 조정하는 것의 중요성.
*   **제한 사항 및 완화:** 자동 평가자의 잠재적 제한 사항과 이를 완화하기 위한 전략을 인식합니다.

이 백서는 LLM을 평가하기 위한 포괄적인 가이드를 제공하며, LLM 기반 애플리케이션을 개발하고 배포하는 사람들에게 유용한 리소스입니다.

**어떤 LLM이 최고인지에 대한 질문에 답하기는 어렵습니다.** "최고"는 특정 사용 사례, 평가 기준 및 우선 순위에 따라 달라지기 때문입니다. 이 백서는 다양한 LLM을 평가하고 선택할 때 고려해야 할 다양한 요소를 이해하는 데 도움이 됩니다.
-----

## 평가

평가 단계:
모델의 응답은 PDF 문서에 대한 요약을 제공해야 합니다. 응답에서 제공된 요약은 독창적이고, 근거가 명확하며, 간결하며, 유창합니다.
이러한 기준을 바탕으로 모델의 응답에 대한 최종 점수는 "매우 좋음"입니다.

점수: 5

-----

SummaryQuality.VERY_GOOD


In [49]:
import functools

terse_guidance = "다음 질문에 한 문장으로, 혹은 가능한 한 그에 가깝게 답하세요."
moderate_guidance = "다음 질문에 대한 간략한 답변을 제공하세요. 필요한 경우 인용을 사용하세요. 하지만 질문에 답하기에 충분한 내용만 제공하세요."
cited_guidance = "다음 질문에 문서를 인용하고 가능한 한 추가 배경 정보를 제공하여 자세하고 자세하게 답변해 주시기 바랍니다."
guidance_options = {
    'Terse': terse_guidance,
    'Moderate': moderate_guidance,
    'Cited': cited_guidance,
}

questions = [
    # 여기에서 시도해 볼 하나 이상의 질문의 주석 처리를 해제하거나 직접 추가하세요.
    # 더 많은 질문을 평가하면 시간이 더 오래 걸리지만, 더 높은 신뢰도로 결과를 얻을 수 있습니다.
    # 프로덕션 시스템에서는 복잡한 시스템을 평가하기 위해 수백 개의
    # 질문이 있을 수 있습니다.

    "장기 컨텍스트 성능을 평가하는 데 사용되는 지표는 무엇입니까?",
    "모델은 코드 작업에서 어떤 성능을 보입니까?",
    "모델은 몇 개의 레이어로 구성되어 있습니까?",
    "제미니라는 이름은 왜 붙었습니까?",
    "자동평가자의 장단점은 무엇입니까?"
]

@functools.cache
def answer_question(question: str, guidance: str = '') -> str:
    """Generate an answer to the question using the uploaded document and guidance."""
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=types.GenerateContentConfig(
            temperature=0.0,
            system_instruction=guidance
        ),
        contents=[question, document_file]
    )
    return response.text


In [53]:
answer = answer_question(questions[4], terse_guidance)
Markdown(answer)

자동 평가자는 확장성과 효율성을 제공하지만 편향이 있을 수 있고 일관성이 부족할 수 있습니다.

In [54]:
import enum

QA_PROMPT = """\
# 지침
귀하는 전문 평가자입니다. AI 모델에서 생성된 응답의 품질을 평가하는 것이 귀하의 임무입니다.
사용자 프롬프트와 AI가 생성한 응답을 제공해 드립니다.
작업 분석을 위해 먼저 사용자 프롬프트를 주의 깊게 읽고, 아래 평가 섹션에 제공된 규칙에 따라 응답의 품질을 평가해야 합니다.

# 평가
## 지표 정의
귀하는 사용자 프롬프트의 질문에 대한 답변의 전반적인 품질을 측정하는 질의응답 품질을 평가합니다. X단어 또는 Y문장과 같은 길이 제한에 특히 유의하십시오. 질의응답 과제 수행 지침은 사용자 프롬프트에 제공됩니다. 응답에는 맥락에 없는 정보가 포함되어서는 안 됩니다(제공된 경우).

채점 기준 및 평가 단계에 따라 작문 응답에 5점, 4점, 3점, 2점, 1점의 점수를 부여합니다.
채점 기준을 단계별로 설명하고 5, 4, 3, 2, 1점 중 하나만 고르세요.

## 기준 정의
지시 준수: 응답은 과제 지시에 대한 명확한 이해를 보여주며, 지시 사항의 모든 요건을 충족합니다.
근거성: 응답은 맥락이 사용자 프롬프트에 존재하는 경우에만 맥락에 포함된 정보를 포함합니다. 응답은 외부 정보를 참조하지 않습니다.
완전성: 응답은 충분한 세부 정보를 포함하여 질문에 완벽하게 답변합니다.
유창성: 응답은 잘 구성되었고 읽기 쉽습니다.

## 평가 기준
5: (매우 좋음). 응답은 지시 사항을 따르고, 근거가 있으며, 완전하고, 유창합니다.
4: (좋음). 응답은 지시 사항을 따르고, 근거가 있으며, 완전하지만, 유창하지는 않습니다.
3: (보통). 응답은 대체로 지시 사항을 따르고, 근거가 있으며, 질문에 부분적으로 답변하지만, 유창하지는 않습니다.
2: (나쁨). 답변이 지시 사항을 잘 따르지 않거나, 불완전하거나, 완전히 근거가 없습니다.
1: (매우 나쁨). 답변이 지시 사항을 따르지 않고, 틀렸으며, 근거가 없습니다.

## 평가 단계
1단계: 기준에 따라 지시 준수, 근거성, 완전성, 유창성 측면에서 답변을 평가합니다.
2단계: 루브릭에 따라 점수를 매깁니다.

# 사용자 입력 및 AI 생성 응답
## 사용자 입력
### 프롬프트
{prompt}

## AI 생성 응답
{response}
"""

class AnswerRating(enum.Enum):
    VERY_GOOD = '5'
    GOOD = '4'
    OK = '3'
    BAD = '2'
    VERY_BAD = '1'


@functools.cache
def eval_answer(prompt, ai_response, n=1):
    """Evaluate the generated answer against the prompt/question used."""
    chat = client.chats.create(model='gemini-2.0-flash')

    # Generate the full text response.
    response = chat.send_message(
        message=QA_PROMPT.format(prompt=[prompt, document_file], response=ai_response)
    )
    verbose_eval = response.text

    # Coerce into the desired structure.
    response = chat.send_message(
        message="Convert the final score.",
        config=types.GenerateContentConfig(
            response_mime_type="text/x.enum",
            response_schema=AnswerRating,
        )
    )
    sturctured_eval = response.parsed

    return verbose_eval, sturctured_eval

In [55]:
text_eval, struct_eval = eval_answer(prompt=questions[4], ai_response=answer)
display(Markdown(text_eval))
print(struct_eval)

평가:

지침 준수: 응답은 지침을 준수합니다.
근거성: 응답은 문서에 있는 정보를 기반으로 합니다.
완전성: 응답은 사용자 질문에 완전하게 답합니다.
유창성: 응답은 잘 구성되고 읽기 쉽습니다.

채점:
5.

AnswerRating.VERY_GOOD


In [56]:
import collections
import itertools

# 오류를 줄이고 평균을 계산하기 위해 각 작업을 반복해야 하는 횟수입니다.
# 반복 횟수를 늘리면 시간은 더 오래 걸리지만 더 나은 결과를 얻을 수 있습니다. 처음에는 2~3회 반복하는 것이 좋습니다.
NUM_ITERATIONS = 1

scores = collections.defaultdict(int)
responses = collections.defaultdict(list)

for question in questions:
    display(Markdown(f'## {question}'))
    for guidance, guide_prompt in guidance_options.items():

        for n in range(NUM_ITERATIONS):
            # Generate a response.
            answer = answer_question(question, guide_prompt)

            # Evaluate the response (note that the guidance prompt is not passed).
            written_eval, struct_eval = eval_answer(question, answer, n)
            print(f'{guidance}: {struct_eval}')

            # Save the numeric score.
            scores[guidance] += int(struct_eval.value)

            # Save the responses, in case you wish to inspect them.
            responses[(guidance, question)].append((answer, written_eval))



## 장기 컨텍스트 성능을 평가하는 데 사용되는 지표는 무엇입니까?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD


## 모델은 코드 작업에서 어떤 성능을 보입니까?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD


## 모델은 몇 개의 레이어로 구성되어 있습니까?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.GOOD


## 제미니라는 이름은 왜 붙었습니까?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD


## 자동평가자의 장단점은 무엇입니까?

Terse: AnswerRating.VERY_GOOD
Moderate: AnswerRating.VERY_GOOD
Cited: AnswerRating.VERY_GOOD


In [57]:
for guidance, score in scores.items():
    avg_score = score / (NUM_ITERATIONS * len(questions))
    nearest = AnswerRating(str(round(avg_score)))
    print(f'{guidance}: {avg_score:.2f} - {nearest.name}')

Terse: 5.00 - VERY_GOOD
Moderate: 5.00 - VERY_GOOD
Cited: 4.80 - VERY_GOOD


In [58]:
QA_PAIRWISE_PROMPT = """\
# 지침
전문가 평가자입니다. 두 AI 모델에서 생성된 응답의 품질을 평가하는 것이 귀하의 임무입니다. 사용자 입력과 AI에서 생성된 응답 쌍(응답 A와 응답 B)을 제공해 드립니다. 먼저 사용자 입력을 주의 깊게 읽고 과제를 분석한 후, 아래 평가 섹션에 제공된 기준에 따라 응답의 품질을 평가해야 합니다.

먼저 평가 기준 및 평가 단계에 따라 개별적으로 응답을 평가합니다. 그런 다음 평가 기준 및 평가 단계에 따라 평가 결과를 비교하여 승자를 결정합니다.

# 평가
## 지표 정의
사용자 프롬프트에 제시된 질문에 대한 답변의 전반적인 품질을 측정하는 질의응답 품질을 평가합니다. X단어 또는 Y문장과 같은 길이 제한에 특히 주의하십시오. 질의응답 과제 수행 지침은 사용자 프롬프트에 제공됩니다. 응답에는 맥락에 없는 정보가 포함되어서는 안 됩니다(제공된 경우).

## 기준
다음 지침: 응답은 질문에 대한 명확한 이해를 보여주며, 과제 지침에 대한 모든 요구 사항을 충족합니다.
근거성: 응답은 맥락이 사용자 프롬프트에 존재하는 경우에만 맥락에 포함된 정보를 포함합니다. 응답은 외부 정보를 참조하지 않습니다.
완전성: 응답은 충분한 세부 정보를 포함하여 질문에 완벽하게 답변합니다.
유창성: 응답은 잘 구성되어 있고 읽기 쉽습니다.

## 평가 기준
"A": 응답 A가 기준에 따라 주어진 질문에 응답 B보다 더 잘 답변합니다.
"동일": 응답 A와 B가 기준에 따라 주어진 질문에 동등하게 잘 답변합니다.
"B": 응답 B가 기준에 따라 주어진 질문에 응답 A보다 더 잘 답변합니다.

## 평가 단계
1단계: 질문 답변 품질 기준에 따라 응답 A를 분석합니다. 응답 A가 사용자 요구 사항을 얼마나 잘 충족하는지, 맥락에 기반을 두고 있는지, 완전하고 유창하며, 기준에 따라 평가를 제공하는지 확인합니다.
2단계: 질문 답변 품질 기준에 따라 응답 B를 분석합니다. 응답 B가 사용자 요구 사항을 얼마나 잘 충족하는지, 맥락에 기반을 두고 있는지, 완전하고 유창하며, 기준에 따라 평가를 제공하는지 확인합니다.
3단계: 분석 및 평가를 기반으로 응답 A와 응답 B의 전반적인 성과를 비교합니다.
4단계: 평가 기준에 따라 "A", "SAME" 또는 "B" 중 선호하는 항목을 pairwise_choice 필드에 출력합니다.
5단계: 설명 필드에 평가 이유를 출력합니다.

# 사용자 입력 및 AI 생성 응답
## 사용자 입력
### 프롬프트
{prompt}

# AI 생성 응답

### 응답 A
{baseline_model_response}

### 응답 B
{response}
"""

In [59]:
class AnswerComparison(enum.Enum):
    A = 'A'
    SAME = 'SAME'
    B = 'B'

@functools.cache
def eval_pairwise(prompt, response_a, response_b, n=1):
    """Determine the better of two answers to the same prompt."""

    chat = client.chats.create(model='gemini-2.0-flash')

    # Generate the full text response.
    response = chat.send_message(
        message=QA_PAIRWISE_PROMPT.format(prompt=[prompt, document_file],
                                          baseline_model_response=response_a,
                                          response=response_b)
    )
    verbose_eval = response.text

    # Coerce into the desired structure.
    response = chat.send_message(
        message="Convert the final score.",
        config=types.GenerateContentConfig(
            response_mime_type="text/x.enum",
            response_schema=AnswerComparison,
        )
    )
    structured_eval = response.parsed

    return verbose_eval, structured_eval

In [61]:
question = questions[0]
answer_a = answer_question(question, terse_guidance)
answer_b = answer_question(question, cited_guidance)

text_eval, struct_eval = eval_pairwise(
    prompt = question,
    response_a=answer_a,
    response_b=answer_b,
)

display(Markdown(text_eval))
print(struct_eval)

# 평가
## 1단계: 응답 A 분석
응답 A는 질문에 직접 답하지 않고 문서에 명시적으로 언급되지 않았다고 말합니다. 이는 도움이 되지 않습니다.

## 2단계: 응답 B 분석
응답 B는 문서에서 다양한 지표를 추출하고 답변했습니다.

## 3단계: 응답 A와 응답 B 비교
응답 B는 응답 A보다 훨씬 낫습니다. 응답 A는 부정적이고 응답 B는 답변에 적절한 정보를 제공합니다.

## 4단계: 선호하는 옵션 출력
B

## 5단계: 설명 출력
응답 B는 파일에 대한 질문을 처리하기 때문에 A보다 훨씬 낫습니다. 응답 A는 답변에 대한 정보가 없다고만 진술합니다.

AnswerComparison.B


In [63]:
display(Markdown(answer_b))

문서에 따르면 장기 컨텍스트 성능을 평가하는 데 사용되는 지표는 다음과 같습니다.

1. **ROUGE(Gisting 평가를 위한 회상 지향적 연구):** n-그램(n-그램 검사) 및 ROUGE-L(가장 긴 공통 하위 시퀀스 기반)과 같은 변형이 있는 메트릭 모음입니다. ROUGE 점수는 0에서 1까지이며 생성된 텍스트와 참조 간의 중복 정도를 나타냅니다.
2. **BLEU(이중 언어 평가 연구):** 원래 기계 번역 평가용으로 설계된 BLEU는 참조와 비교하여 생성된 텍스트에서 n-그램의 정확도를 측정합니다.
3. **BERTScore 및 BARTScore:** 이러한 방법은 단어와 구의 문맥화된 임베딩을 비교하여 응답의 의미론적 유사성을 조사합니다. 이를 통해 표면 수준 일치 이상의 의미에 대한 더 깊은 이해가 가능합니다.

이러한 계산 기반 방법은 모델의 출력과 주어진 참조 간의 유사성을 측정하여 LLM을 평가하는 정량적 접근 방식을 제공합니다. 효율적이고 객관적이지만 본질적인 제한 사항이 있습니다. 단일 점수를 생성하는 pointwise 평가만 지원합니다. 여러 출력에 대한 점수를 비교하고 순위를 매길 수 있지만 이러한 메트릭은 각 응답을 개별적으로 측정한다는 점을 명심하는 것이 중요합니다. 또한 세분화된 기준이나 미묘한 측면을 통합할 수 없습니다.

In [92]:
@functools.total_ordering
class QAGuidancePrompt:
    """A question-answering guidance prompt or system instruction."""

    def __init__(self, prompt, questions, n_comparisons=NUM_ITERATIONS):
        """Create the prompt. Provide questions to evaluate against, and number of evals to perform."""
        self.prompt = prompt
        self.questions = questions
        self.n = n_comparisons

    def __str__(self):
        return self.prompt
    
    def _compare_all(self, other):
        """Compare two prompts on all questions over n trials."""
        results = [self._compare_n(other, q) for q in questions]
        mean = sum(results) / len(results)
        print(f'{results}: round(mean({mean}))={round(mean)}')
        return round(mean)
    
    def _compare_n(self, other, question):
        """Compare two prompts on a question over n trials."""
        results = [self._compare(other, question, n) for n in range(self.n)]
        mean = sum(results) / len(results)
        return mean
    
    def _compare(self, other, question, n=1):
        """Compare two prompts on a single question."""
        answer_a = answer_question(question, self.prompt)
        answer_b = answer_question(question, other.prompt)

        _, result = eval_pairwise(
            prompt=question,
            response_a=answer_a,
            response_b=answer_b,
            n=n,    # Cache buster
        )
        print(f'q[{question}], a[{self.prompt[:20]}...], b[{other.prompt[:20]}...]: {result}')

        # Convert the enum to the standard Python numeric comparison values.
        if result is AnswerComparison.A:
            return 1
        elif result is AnswerComparison.B:
            return -1
        else:
            return 0
        
    def __eq__(self, other):
        """Equality check that performs pairwise evaluation."""
        if not isinstance(other, QAGuidancePrompt):
            return NotImplemented
        
        return self._compare_all(other) == 0
    
    def __lt__(self, other):
        """Ordering check that prrforms pairwise evaluation."""
        if not isinstance(other, QAGuidancePrompt):
            return NotImplemented
        
        return self._compare_all(other) < 0      


In [93]:
terse_prompt = QAGuidancePrompt(terse_guidance, questions)

In [94]:
moderate_prompt = QAGuidancePrompt(moderate_guidance, questions)

In [95]:
cited_prompt = QAGuidancePrompt(cited_guidance, questions)

In [97]:
sorted_results = sorted([terse_prompt, moderate_prompt, cited_prompt], reverse=True)
for i, p in enumerate(sorted_results):
    if i:
         print("---")
    print(f'#{i+1}: {p}')

q[장기 컨텍스트 성능을 평가하는 데 사용되는 지표는 무엇입니까?], a[다음 질문에 대한 간략한 답변을 제공...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.B
q[모델은 코드 작업에서 어떤 성능을 보입니까?], a[다음 질문에 대한 간략한 답변을 제공...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.B
q[모델은 몇 개의 레이어로 구성되어 있습니까?], a[다음 질문에 대한 간략한 답변을 제공...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.B
q[제미니라는 이름은 왜 붙었습니까?], a[다음 질문에 대한 간략한 답변을 제공...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.SAME
q[자동평가자의 장단점은 무엇입니까?], a[다음 질문에 대한 간략한 답변을 제공...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.B
[-1.0, -1.0, -1.0, 0.0, -1.0]: round(mean(-0.8))=-1
q[장기 컨텍스트 성능을 평가하는 데 사용되는 지표는 무엇입니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 대한 간략한 답변을 제공...]: AnswerComparison.A
q[모델은 코드 작업에서 어떤 성능을 보입니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 대한 간략한 답변을 제공...]: AnswerComparison.B
q[모델은 몇 개의 레이어로 구성되어 있습니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 대한 간략한 답변을 제공...]: AnswerComparison.SAME
q[제미니라는 이름은 왜 붙었습니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 대한 간략한 답변을 제공...]: AnswerComparison.SAME
q

In [105]:
sorted([cited_prompt, terse_prompt, ])

q[장기 컨텍스트 성능을 평가하는 데 사용되는 지표는 무엇입니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.B
q[모델은 코드 작업에서 어떤 성능을 보입니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.B
q[모델은 몇 개의 레이어로 구성되어 있습니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.A
q[제미니라는 이름은 왜 붙었습니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.SAME
q[자동평가자의 장단점은 무엇입니까?], a[다음 질문에 한 문장으로, 혹은 가능...], b[다음 질문에 문서를 인용하고 가능한 ...]: AnswerComparison.B
[-1.0, -1.0, 1.0, 0.0, -1.0]: round(mean(-0.4))=0
